# Лабораторная работа №6. Применение сверточных нейронных сетей (многоклассовая классификация)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Задание 1.
Загрузите данные. Разделите исходный набор данных на обучающую и валидационную выборки.

In [23]:
import os
import pandas as pd

dataset_path = os.path.join('data', 'sign-language-mnist')

train_data = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_train.csv'))
test_data = pd.read_csv(os.path.join(dataset_path, 'sign_mnist_test.csv'))

In [24]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [25]:
from sklearn.model_selection import train_test_split

y_train = train_data['label'].values
train_data.drop(columns='label', inplace=True)
X_train = train_data.values

y_test = test_data['label'].values
test_data.drop(columns='label', inplace=True)
X_test = test_data.values

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    stratify=y_train
)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

## Задание 2.
Реализуйте глубокую нейронную сеть со сверточными слоями. Какое качество классификации получено? Какая архитектура сети была использована?

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Dropout, Flatten

network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(24, activation='softmax')
])

network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_13 (Batc (None, 12, 12, 64)        256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 64)         

In [31]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_data_generator(X, y):
    gen = ImageDataGenerator(rescale=1/255.)
    return gen.flow(
        X,
        to_categorical(y),
        batch_size=64,
        seed=42,
        shuffle=True,
    )

train_gen = get_data_generator(X_train, y_train)
val_gen = get_data_generator(X_val, y_val)
test_gen = get_data_generator(X_test, y_test)

In [32]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

network.fit(
    train_gen, 
    epochs=15,
    validation_data=val_gen,
    workers=4,
)